# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from Config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [37]:
#CSV
chart_data = pd.read_csv('../output_data/weather_city.csv')
vac_df = pd.DataFrame(chart_data)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [39]:
# Use the Lat and Lng as locations 
locations = vac_df[["lat", "lon"]]

# Humidity as the weight
humidity = vac_df["humidity"].astype(float)

In [44]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
#locate vacation spots in the Philippians
desired_df = vac_df.loc[(vac_df["max temp"] < 75) & (vac_df["max temp"] > 70) & \
                       (vac_df["wind_speed"] < 10) & (vac_df["cloudiness"] == 0)]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
# set up additional columns to hold information
desired_df["hotel name"] = ""
desired_df.head()

c:\users\simo8\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,lat,lon,max temp,humidity,cloudiness,wind_speed,country,date,hotel name
27,vigrestad,-23.12,-134.97,73.80,78,0,5.19,PF,1596334142,
90,albany,-8.93,125.41,70.93,38,0,4.29,TL,1596334158,
209,west wendover,-19.01,-57.65,74.62,38,0,2.06,BR,1596334364,
238,lompoc,63.92,127.47,73.80,41,0,4.09,RU,1596334371,
353,pilar,47.87,88.12,72.39,30,0,2.93,CN,1596334399,


In [48]:
# # Store 'Lat' and 'Lng' into  locations 
# locations = desired_df[["lat", "lon"]].astype(float)

desired_df_complete = desired_df.dropna()

In [9]:
# #Test Dataframe of two lat lng coordinates
# lat_lng_test_df = pd.DataFrame({"lat": [49.14, 36.56], "lon": [56.49, 104.21]})


# # complete_list= lat_df + lon_df

# # use iterrows to iterate through pandas dataframe
# for index, row in lat_lng_test_df.iterrows():
    
#     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     params = {
#         "location": f"{row[0]}, {row[1]}", 
#         "rankby": "distance",
#         "type": "lodging",
#         "key": g_key,
#         }

#     # assemble url and make API request
#     response = requests.get(base_url, params=params).json()
#     print(json.dumps(response, indent = 3))
    
#     # extract results
#     results = response['results']
    
#     try:    
#         lat_lng_test_df.loc[index, 'lodging'] = results[0]['name']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
# lat_lng_test_df

In [49]:
# find the closest hotel of each type to coordinates
#https://developers.google.com/places/web-service/supported_types
#https://developers.google.com/places/web-service/search#PlaceSearchRequests

# use iterrows to iterate through pandas dataframe
for index, row in desired_df_complete.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{row[1]}, {row[2]}", 
        "rankby": "distance",
        "type": "lodging",
        "key": g_key,
        }

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
#     print(json.dumps(response, indent = 3))
    
    # extract results
    results = response['results']
    
    try:    
        desired_df_complete.loc[index, 'hotel name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [50]:
#clean up result
#resource: https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
desired_df_complete['hotel name'].replace('', np.nan, inplace=True)
desired_df_complete
hotel_df = desired_df_complete.dropna()
hotel_df

,city,lat,lon,max temp,humidity,cloudiness,wind_speed,country,date,hotel name
27,vigrestad,-23.12,-134.97,73.80,78,0,5.19,PF,1596334142,Pension Maro'i
90,albany,-8.93,125.41,70.93,38,0,4.29,TL,1596334158,Juvinal Martin
209,west wendover,-19.01,-57.65,74.62,38,0,2.06,BR,1596334364,Hotel El Shadday
353,pilar,47.87,88.12,72.39,30,0,2.93,CN,1596334399,陶然家园
396,buenaventura,39.05,-27.98,70.92,70,0,8.41,PT,1596334410,Camping's Pine Park
473,tahta,36.56,104.21,73.40,28,0,6.71,CN,1596334428,Super 8 Hotel Baiyin Jing Tai Square
486,flin flon,-17.78,-57.77,73.54,41,0,1.59,BO,1596334431,Hotel palance
487,anderson,49.63,-105.98,73.27,39,0,7.72,CA,1596334431,Bar B Motel


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [52]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations)

# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Create a combined map
# fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…